In [1]:
from statsforecast.adapters.prophet import AutoARIMAProphet
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

import warnings
warnings.filterwarnings("ignore")

2022-07-08 13:39:01 prophet.plot ERROR: Importing plotly failed. Interactive plots will not work.


In [2]:
n_sys = 10
n_test = 24
n_data = 10000

In [3]:
data =  pd.read_csv('../../../Data/pv_power_df_5day_capacity_scaled.csv', index_col='datetime').drop(columns=['2657', '2828']) #DROPPING FAULTY SYSTEMS
uk_pv = pd.read_csv('../../../Data/system_metadata_location_rounded.csv')
uk_pv['ss_id_string'] = uk_pv['ss_id'].astype('str')

data_multiple = data.iloc[:, :n_sys][:n_data].reset_index()
capacities = uk_pv[uk_pv.ss_id_string.isin(data_multiple.columns)].set_index('ss_id_string')['kwp'].values * 1000


## Step 1
First I run the autoarima on a subsample of the models, see what model gets chosen most often.

In [17]:
start = time.time()
models = dict.fromkeys(data.columns[:n_sys])

for idx_series, system_id in enumerate(data.columns[:n_sys]):
    print(f'constructing time series for system {system_id}')
    ts = data.iloc[:n_data, idx_series].to_frame()
    ts = ts.T.stack().to_frame().reset_index().set_index('level_0')
    ts = ts.rename(index={'level_0':'unique_id'}, columns = {'datetime':'ds', 0:'y'})
    ts.index = ts.index.rename('unique_id')

    #following this to put constant trend: https://github.com/facebook/prophet/issues/614
    print(f'defining the model')
    m = AutoARIMAProphet(growth = 'flat', yearly_seasonality = True, weekly_seasonality =False, daily_seasonality = True,
                            seasonality_mode = 'multiplicative', 
                            d=1,
                            D=1,
                            max_p=2,
                            max_q=2,
                            max_P=2,
                            max_Q=2,
                            start_p=1,
                            start_q=1,
                            start_P=1,
                            start_Q=1,
                            stationary=False,
                            seasonal=True,
                            ic='aicc',
                            stepwise=True,
                            nmodels=40,
                            test='kpss',
                            seasonal_test='seas',
                            num_cores=None,
                            period=97,
                            approximation = True,
    )
    print('fit the model')
    m.fit(ts)
    print('save the data in a dict')
    models[system_id] = m.arima.model_
    
end =  time.time()
time_loop = end - start
print(f'time it took is {time_loop}')

constructing time series for system 2607
defining the model
fit the model
save the data in a dict
constructing time series for system 2625
defining the model
fit the model
save the data in a dict
constructing time series for system 2626
defining the model
fit the model
save the data in a dict
constructing time series for system 2631
defining the model
fit the model
save the data in a dict
constructing time series for system 2660
defining the model
fit the model
save the data in a dict
constructing time series for system 2729
defining the model
fit the model
save the data in a dict
constructing time series for system 2760
defining the model
fit the model
save the data in a dict
constructing time series for system 2766
defining the model
fit the model
save the data in a dict
constructing time series for system 2770
defining the model
fit the model
save the data in a dict
constructing time series for system 2775
defining the model
fit the model
save the data in a dict
time it took is 1321

## Step 2

Check which model is chosen most often

In [18]:
models

{'2607': ARIMA(2,1,2)(0,1,0)[97]                   ,
 '2625': ARIMA(2,1,2)(0,1,0)[97]                   ,
 '2626': ARIMA(1,1,2)(0,1,0)[97]                   ,
 '2631': ARIMA(2,1,2)(0,1,0)[97]                   ,
 '2660': ARIMA(2,1,2)(0,1,0)[97]                   ,
 '2729': ARIMA(2,1,2)(0,1,0)[97]                   ,
 '2760': ARIMA(2,1,2)(0,1,0)[97]                   ,
 '2766': ARIMA(1,1,2)(0,1,0)[97]                   ,
 '2770': ARIMA(2,1,2)(0,1,0)[97]                   ,
 '2775': ARIMA(2,1,2)(0,1,0)[97]                   }

## Step 3

- Train the most popular model on the whole data for the first 2 years (this way we don't need to do autoarima)

The most popular model was ARIMA(2,1,2)(0,1,0)[97] 

- Perform Cross Validation on these models, using the builtin function


- Add also the features to convert the MAE into MW units

In [4]:
start = time.time()
models = dict.fromkeys(data.columns[:n_sys])
models_params = dict.fromkeys(data.columns[:n_sys])
# df_p_tot = dict.fromkeys(data.columns[:n_sys])
# df_cv_tot = dict.fromkeys(data.columns[:n_sys])

for idx_series, system_id in enumerate(data.columns[:n_sys]):
    print(f'constructing time series for system {system_id}')
    ts = data.iloc[:n_data, idx_series].to_frame()
    ts = ts.T.stack().to_frame().reset_index().set_index('level_0')
    ts = ts.rename(index={'level_0':'unique_id'}, columns = {'datetime':'ds', 0:'y'})
    ts.index = ts.index.rename('unique_id')

    #following this to put constant trend: https://github.com/facebook/prophet/issues/614
    print(f'defining the model')
    m = AutoARIMAProphet(growth = 'flat', yearly_seasonality = True, weekly_seasonality =False, daily_seasonality = True,
                            seasonality_mode = 'multiplicative', 
                            d=1,
                            D=1,
                            max_p=2,
                            max_q=2,
                            max_P=0,
                            max_Q=0,
                            start_p=2,
                            start_q=2,
                            stationary=False,
                            seasonal=True,
                            ic='aicc',
                            stepwise=True,
                            nmodels=5,
                            test='kpss',
                            seasonal_test='seas',
                            num_cores=None,
                            period=97,
                            approximation = True,
                            trace=True                  
    )
    print('fit the model')
    m.fit(ts)

    print('save the data in a dict')
    models[system_id] = m
    models_params[system_id] = m.arima.model_

end =  time.time()
time_loop = end - start
print(f'time it took is {time_loop}')

constructing time series for system 2607
defining the model
fit the model
Fitting models using approximations to speed things up

ARIMA(2,1,2)(0,1,0)[97]                   :-14355.002561757196

ARIMA(0,1,0)(0,1,0)[97]                   :-13555.17042403563

ARIMA(1,1,0)(0,1,0)[97]                   :-13683.367419417567

ARIMA(0,1,1)(0,1,0)[97]                   :-13767.806204148312

ARIMA(1,1,2)(0,1,0)[97]                   :-14295.054262733587
Now re-fitting the best model(s) without approximations...


ARIMA(2,1,2)(0,1,0)[97]                   :-14867.90504023886
save the data in a dict
time it took is 148.4996371269226


## Step 4
- Tesitng the model without retraining


from: https://stackoverflow.com/questions/66870279/arima-forecasting-next-steps-without-updating-model

In [5]:
data_index = data.index 
data_index = pd.to_datetime(data.index)

## Statsmodels ARIMA is too slow!
Instead I will try to get the model with statsforecasts, and then pass it to statsmodels

In [9]:
def rename_keys(d, keys):
    return dict([(keys.get(k), v) for k, v in d.items()])


In [38]:
import copy

M = models['2607']
model_coeff = M.arima.model_.model['coef']
model_coefficient = copy.deepcopy(model_coeff)
mapping = {'ma1':'ma.L1', 'ma2':'ma.L2', 'ar1':'ar.L1', 'ar2':'ar.L2', 'sigma2':'sigma2'}
model_coefficient = rename_keys(model_coefficient, mapping)

#setting the sigma to 0
model_coefficient['sigma2'] = M.arima.model_.model['sigma2']

In [19]:
# model_coefficient['ar1'] = model_coefficient.pop('ar.L.L1')
# model_coefficient['ar2'] = model_coefficient.pop('ar.L.L2')

In [41]:
from statsmodels.tsa.arima.model import ARIMA

start = time.time()
series = data.iloc[:,0].reset_index().drop(columns='datetime')
# Split data into test and training
nobs = len(series)
n_train = 10000
series_train = series.iloc[:n_train]

# Fit the model using the training dataset
p, q, P, Q, per, d, D = [M.arima.model_.model['arma'][i]  for i in range(7)] #order=(2,1,2), seasonal_order=(0,1,0,97)

#USING THE PARAMETERS FROM THE MODEL, AND FIXING THE INITIAL PARAMETERS
model = ARIMA(series_train, order=(p,d,q), seasonal_order=(P,D,Q,per))
with model.fix_params(model_coefficient):
        fit = model.fit()

end =  time.time()
time_loop = end - start
print(f'time it took is {time_loop}')

time it took is 42.88771462440491


In [49]:
series_train

,2607
0,0.000000
1,0.000000
2,0.000000
3,0.000000
4,0.000000
...,...
9995,0.324518
9996,0.333120
9997,0.379733
9998,0.321640


In [51]:
series.iloc[t:t + 2]

,2607
10000,0.266683
10001,0.268018


In [48]:
res.append()

In [52]:
start = time.time()

# Compute the first forecast based only on the training dataset
forecasts = []
res = fit
forecasts.append(res.forecast(12))

# Now step through the test observations:
# (a) add the new observation without refitting the model
# (b) produce a new forecast
for t in range(n_train, n_train+20):#nobs):
    
    # REMEMBER TO NOT FORECAST ON THE FIRST 12 TIMESTEPS, AND ON THE NEXT DAY WORTH OF DATA
    print('doing step',t)
    # Update the results by appending the next observation
    res = res.append(series.iloc[t:t + 2], refit=False)
    # Produce a forecast for t+1 based on data through t
    print('forecasting')
    forecasts.append(res.forecast(12))
    print('performed forecasts')
    
end =  time.time()
time_loop = end - start
print(f'time it took is {time_loop}')

doing step 10000


ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
#https://stackoverflow.com/a/44559180/2901002
def justify(a, invalid_val=0, axis=1, side='left'):    
    """
    Justifies a 2D array

    Parameters
    ----------
    A : ndarray
        Input array to be justified
    axis : int
        Axis along which justification is to be made
    side : str
        Direction of justification. It could be 'left', 'right', 'up', 'down'
        It should be 'left' or 'right' for axis=1 and 'up' or 'down' for axis=0.

    """

    if invalid_val is np.nan:
        mask = ~np.isnan(a)
    else:
        mask = a!=invalid_val
    justified_mask = np.sort(mask,axis=axis)
    if (side=='up') | (side=='left'):
        justified_mask = np.flip(justified_mask,axis=axis)
    out = np.full(a.shape, invalid_val) 
    if axis==1:
        out[justified_mask] = a[mask]
    else:
        out.T[justified_mask.T] = a.T[mask.T]
    return out

In [ ]:
#THIS CODE CREATED A DATASET OF OBSERVATIONS OVER TIME, compresses it, and calculates the error as function of horizon
df_preds = pd.concat(forecasts,axis=1)
arr = justify(df_preds.T.to_numpy(), invalid_val=np.nan)
df = pd.DataFrame(arr).dropna(axis=1, how='all').T
forecast_error = (df.T - series[100:121].values).mean(axis=0)

## Step X
- Once I work on the cross validation for the Gaussian Process, use the same methodology for this ARIMA model


In [ ]:
start = time.time()
# models = dict.fromkeys(data.columns[:n_sys])
# models_params = dict.fromkeys(data.columns[:n_sys])
# preds = dict.fromkeys(data.columns[:n_sys])

# for idx_series, system_id in enumerate(data.columns[:n_sys]):
#     print(f'constructing time series for system {system_id}')
#     ts_test = data.iloc[n_data:, idx_series].to_frame()
#     ts = ts_test.T.stack().to_frame().reset_index().set_index('level_0')
#     ts_test = ts_test.rename(index={'level_0':'unique_id'}, columns = {'datetime':'ds', 0:'y'})
#     ts_test.index = ts_test.index.rename('unique_id')

#     model_fit = models[system_id]
    
#     print('create the future df')
#     day_max = 16
#     day_min = 8
#     M = models['2607']
#     future = M.make_future_dataframe(periods=1000, freq = '5min', include_history=False)
#     future2 = future[(future.ds.dt.hour >= 8)  & (future.ds.dt.hour < 16) ]
#     future2 = future2.iloc[:n_test]
#     print('forecast')
#     forecast = M.predict(future2).dropna(subset='ds')
    
    
# end =  time.time()
# time_loop = end - start
# print(f'time it took is {time_loop}')

In [ ]:
#     print('create the future df')
#     day_max = 16
#     day_min = 8
#     future = m.make_future_dataframe(periods=1000, freq = '5min')
#     future2 = future[(future.ds.dt.hour >= 8)  & (future.ds.dt.hour < 16) ]
#     future2 = future2.iloc[:len(ts) + n_test]

#     print('forecast')
#     forecast = m.predict(future2).dropna(subset='ds')
    
#     print('perform CV')
#     init = str(int((n_data/97) / 2)) + ' days' #start midway trough the ts
# #     init = '400 days' #give enough time to have a full year of data for seasonality
#     period = str(max(1, int((n_data/97) / 2 / 35))) + ' days ,  2 hours' #divide into 50 predictions, add 2 hours
#     df_cv = cross_validation(m, initial=init, period=period, horizon = '2 hours')
#     df_p = performance_metrics(df_cv)


#     df_p_tot[system_id] = df_p
#     df_cv_tot[system_id] = df_cv

In [ ]:
# accuracy_df = pd.concat(df_p_tot)
# mean_accuracy_result = accuracy_df.droplevel(1).groupby(axis=0, level=0).mean() 
# models_df = pd.DataFrame.from_dict(models, orient='index')
# maes = (mean_accuracy_result.T * capacities).loc['mae']


# mean_accuracy_result.to_csv('accuracies')
# models_df.to_csv('models')
# maes.to_csv('maes')


In [ ]:
# from prophet.plot import plot_cross_validation_metric
# fig = plot_cross_validation_metric(df_cv, metric='mae')

# fig1 = m.plot(forecast,)
# plt.xlim(forecast.ds[800], forecast.ds[1023])
# m.arima.summary()